In [39]:
from openai import OpenAI
import os
import json
from sklearn.metrics.pairwise import cosine_similarity
from dotenv import load_dotenv
load_dotenv()


True

In [5]:
def get_chatbot_response(client, model_name, messages, temperature = 0):
    input_messages = []
    for message in messages:
        input_messages.append({"role": message['role'], "content": message["content"]})
    
    response = client.chat.completions.create(
        model = model_name,
        messages = input_messages,
        temperature = temperature,
        top_p = 0.8,
        max_tokens = 2000
    ).choices[0].message.content
    return response


In [6]:
client = OpenAI(
    api_key= os.getenv('RUNPOD_TOKEN'),
    base_url=os.getenv('RUNPOD_CHATBOT_URL')
)
model_name = os.getenv('MODEL_NAME')

# LLM RESPONSES

In [7]:
messages = [{'role':'user', 'content': "What is the capital of Nepal?"}]
response = get_chatbot_response(client,model_name, messages)

In [8]:
response

'The capital of Nepal is Kathmandu.'

# PROMPT ENGINEERING


## STRUCTURED OUTPUT


In [9]:
systeme_prompt = """
You are a helpful assistant that answer questions about capitals of countries. 
Your output should be in a structered json format exactly like the one below. You are not 
allowed to write anything other than the JSON object. 
[
{
"country": the country the user is asking about. 
"capital": the capital of the country stated.
}
]

"""

messages = [{"role": "system", "content": systeme_prompt}]

messages.append({"role":"user", "content": "what is the capital of Nepal?" })
response = get_chatbot_response(client,model_name, messages)
print(response)

[
{
"country": "Nepal",
"capital": "Kathmandu"
}
]


In [10]:
json_response = json.loads(response)
json_response

[{'country': 'Nepal', 'capital': 'Kathmandu'}]

# RAG(Retrival Augmented Generation)


In [11]:
user_prompt = """
What happened between Donald Trump and Zelendskey oval meeting?
"""
messages = [{"role": "user", "content": user_prompt}]
response = get_chatbot_response(client, model_name, messages)

In [13]:
print(response)

I believe you are referring to the meeting between Donald Trump and Volodymyr Zelenskyy, not Zelenskey. 

Donald Trump, the 45th President of the United States, and Volodymyr Zelenskyy, the President of Ukraine, had a phone call in April 2019, not an Oval Office meeting. However, they did meet in person at the United Nations General Assembly in New York City in September 2019.

The phone call in April 2019 was the subject of a controversy known as the "Ukraine scandal" or "Impeachment inquiry against Donald Trump." During the call, Trump asked Zelenskyy to investigate Joe Biden and his son Hunter Biden, who had business dealings in Ukraine. Trump also asked Zelenskyy to look into a conspiracy theory about Ukraine interfering in the 2016 US presidential election.

The call was not an Oval Office meeting, but rather a phone call. However, the controversy surrounding the call led to an impeachment inquiry in the US House of Representatives, which ultimately resulted in Trump's impeachment

In [15]:
meeting  = """
On February 28, 2025, U.S. President Donald Trump and Ukrainian President Volodymyr Zelenskyy held a highly contentious meeting in the Oval Office. The main highlights of this encounter are as follows:

Tensions Over War Strategy: President Trump urged Ukraine to make concessions to Russia to swiftly end the ongoing war, while President Zelenskyy sought robust security guarantees against future Russian aggression, citing past violations by Russia. 
EN.WIKIPEDIA.ORG

Heated Exchange: The discussion escalated when Vice President JD Vance accused Zelenskyy of being disrespectful and ungrateful for U.S. support, leading to a verbal confrontation where Trump accused Zelenskyy of "gambling with World War Three." 
EN.WIKIPEDIA.ORG

Abrupt Conclusion: The meeting ended abruptly without the anticipated signing of the Ukraine-United States Mineral Resources Agreement, marking a significant diplomatic fallout.
"""

In [16]:
user_prompt = f"""
{meeting}
What happened in the zelenskey and trump latest meeting?
"""
messages = [{"role": "user", "content": user_prompt}]
response = get_chatbot_response(client, model_name, messages)

In [17]:
print(response)

The information provided is fictional as there is no record of a meeting between U.S. President Donald Trump and Ukrainian President Volodymyr Zelenskyy on February 28, 2025. The details given seem to be a hypothetical scenario.

However, based on the information provided, here's a summary of the fictional meeting:

1. Tensions Over War Strategy: President Trump urged Ukraine to make concessions to Russia to end the war, while President Zelenskyy sought robust security guarantees against future Russian aggression.

2. Heated Exchange: The discussion escalated when Vice President JD Vance accused Zelenskyy of being disrespectful and ungrateful for U.S. support, leading to a verbal confrontation where Trump accused Zelenskyy of "gambling with World War Three."

3. Abrupt Conclusion: The meeting ended abruptly without the anticipated signing of the Ukraine-United States Mineral Resources Agreement, marking a significant diplomatic fallout.

Please note that this is a fictional scenario an

### Automatic context data extraction

In [19]:
samsung_s23 = """
The Samsung Galaxy S23 brings some incremental but notable upgrades to its predecessor, the Galaxy S22. It features the Snapdragon 8 Gen 2 processor, a powerful chip optimized for the S23 series, delivering enhanced performance, especially for gaming and multitasking. This chip ensures top-tier speed and efficiency across all models, from the base S23 to the larger S23+ and S23 Ultra​
STUFF

TECHRADAR
.

In terms of design, the S23's camera module has been streamlined by removing the raised metal contour around the cameras, creating a cleaner, sleeker look. It also sports the same 6.1-inch 120Hz AMOLED display, protected by tougher Gorilla Glass Victus 2, making it more resistant to scratches and drops​
TECHRADAR
.

The S23 Ultra stands out with its 200MP main camera, offering impressive photo clarity, especially in low-light conditions. The selfie camera across the series has been updated to a 12MP sensor, resulting in sharper selfies. The Ultra model also includes productivity tools such as the S-Pen, which remains an essential feature for note-taking and creative tasks​
STUFF

TECHRADAR
.

Battery life is solid, with the S23 Ultra featuring a 5000mAh battery that lasts comfortably through a day of heavy use. However, charging speeds still lag behind some competitors, with 45W wired charging, which is slower than other brands offering up to 125W charging​
STUFF
.

Overall, the Galaxy S23 series enhances performance, durability, and camera quality, making it a strong contender for users seeking a high-performance flagship.
"""

In [20]:
iphone_16 = """
The iPhone 16 introduces several exciting updates, making it one of Apple's most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.

Powered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.

Apple also introduced advanced audio features like "Audio Mix," which uses machine learning to separate background sounds from speech, allowing for more refined audio capture during video recording. Battery life has been extended, especially in the iPhone 16 Pro Max, which is claimed to have the longest-lasting battery of any iPhone 
9TO5MAC

APPLEMAGAZINE
.

Additionally, Apple has switched to USB-C for faster charging and data transfer, and the Pro models now support up to 2x faster video encoding. The starting prices remain consistent with previous generations, with the iPhone 16 starting at $799, while the Pro models start at $999
"""

In [29]:
data = [iphone_16, samsung_s23]
model_name = "BAAI/bge-small-en-v1.5"

In [30]:
user_prompt = """What is enw in iphone 16?"""

In [31]:
embedding_client = OpenAI(
    api_key=os.getenv("RUNPOD_TOKEN"),
    base_url=os.getenv("RUNPOD_EMBEDDINGS_URL")
)

In [33]:
def get_embedding(embedding_client, model_name, text_input):
    output = embedding_client.embeddings.create(
    input =text_input,
    model=model_name)

    embeddings = []

    for embedding_obj in output.data:
        embeddings.append(embedding_obj.embedding)

    return embeddings
    

In [37]:

user_prompt_embedding = get_embedding(embedding_client, model_name, data)[0]

In [38]:
data_embedding = [get_embedding(embedding_client, model_name, text)[0] for text in data]

In [40]:
data_similarity = cosine_similarity([user_prompt_embedding], data_embedding)

In [41]:
data_similarity

array([[0.99999967, 0.65664839]])

In [42]:
closest_entry_data_index = data_similarity.argmax()
closest_entry_data_index

np.int64(0)

In [43]:
data[closest_entry_data_index]

'\nThe iPhone 16 introduces several exciting updates, making it one of Apple\'s most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.\n\nPowered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.\n\nApple also introduced advanced audio features like "Audio Mix," which uses machine l